# Clone Groups
*Jupyter Notebook to clone groups and their items*

In [ ]:
# common imports
from arcgis.gis import GIS

## User Input

### GIS Configuration
Parameter information [here](https://developers.arcgis.com/python/guide/using-the-gis/).
* **template** - the ArcGIS Online organization that contains the default groups & items
* **project** - the organization used by the AEC firm to work on the project
* **delivery_orgs** - an array of organizations used to deliver project work to 3rd parties

In [ ]:
# log in to respective portals
template = GIS("<template GIS>", "<un>", "<pw>")
project = GIS("<project GIS>", "<un>", "<pw>")
delivery_orgs = [GIS("<delivery GIS>", "<un>", "<pw>")]

### Clone Configuration
Defines the groups that will be cloned and the naming structures of cloned items & groups

In [ ]:
# groups to be copied from template to project organization
PROJECT_GROUPS = ["data", "maps", "internal apps"]
# groups to be copied from template to delivery organization
DELIVERY_GROUPS = ["external apps"]

# new item name structure, {} replaced by item name
NAME_STRUCTURE = "<PROJECT NAME> {}"
# user folder for new items, as is creates string "<PROJECT NAME> Content"
FOLDER = NAME_STRUCTURE.format("Content")

### Utility Functions
**NOTE** if you are unable to import local files, **replace the following cell** with the code in [**clone_utils.py**](https://github.com/ArcGIS/aec-scripts/blob/master/utils/clone_utils.py)

In [ ]:
from utils.clone_utils import search_group_title, search_item_title, clone_items_modify

## Execution

### Modify Functions
*functions to modify the title of the cloned items & groups*

In [ ]:
# update title functions
def modify_item(item, target):
    title = NAME_STRUCTURE.format(item.title)
    while search_item_title(target, title):
        title = input("Title `{0}` for ITEM `{1}` already exists \n new title: "
                         .format(title, item.title))
    return {"title": title}
def modify_group(group, expected_title, target):
    title = NAME_STRUCTURE.format(expected_title)
    while search_group_title(target, title):
        title = input("Title `{0}` for GROUP `{1}` already exists \n new title: "
                         .format(title, expected_title))
    return {"title": title}

### Clone

In [ ]:
# get groups
template_project_groups = [search_group_title(template, title) for title in PROJECT_GROUPS]
template_delivery_groups = [search_group_title(template, title) for title in DELIVERY_GROUPS]

# copy the groups
project_results = clone_items_modify(template_project_groups, project,
                                     modify_item_callback=modify_item,
                                     modify_group_callback=modify_group,
                                     copy_data=False, search_existing_items=False, folder=FOLDER)

delivery_results = []
for delivery in delivery_orgs:
    results = clone_items_modify(template_delivery_groups, delivery,
                                 modify_item_callback=modify_item,
                                 modify_group_callback=modify_group,
                                 copy_data=False, search_existing_items=False, folder=FOLDER)
    delivery_results += results